In [8]:
%pip install --upgrade -q aiobotocore

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import boto3
import sagemaker
import time
import json
import io
from io import StringIO
import base64
import pprint
import re

from sagemaker.image_uris import retrieve

sess = sagemaker.Session()
write_bucket = sess.default_bucket()
write_prefix = "fraud-detect-demo"

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

sagemaker_role = sagemaker.get_execution_role()


# S3 locations used for parameterizing the notebook run
read_bucket = "sagemaker-sample-files"
read_prefix = "datasets/tabular/synthetic_automobile_claims" 
model_prefix = "models/xgb-fraud"

data_capture_key = f"{write_prefix}/data-capture"

# S3 location of trained model artifact
model_uri = f"s3://{read_bucket}/{model_prefix}/fraud-det-xgb-model.tar.gz"

# S3 path where data captured at endpoint will be stored
data_capture_uri = f"s3://{write_bucket}/{data_capture_key}"

# S3 location of test data
test_data_uri = f"s3://{read_bucket}/{read_prefix}/test.csv" 

In [11]:
print(f' sagemaker_role : {sagemaker_role}')

 sagemaker_role : arn:aws:iam::720025708473:role/CFN-SM-IM-Lambda-Catalog-SageMakerExecutionRole-XRg8SB5ffteM


In [13]:
# Retrieve the SageMaker managed XGBoost image
training_image = retrieve(framework="xgboost", region=region, version="1.3-1")

# Specify a unique model name that does not exist
model_name = "fraud-detect-xgb"
primary_container = {
                     "Image": training_image,
                     "ModelDataUrl": model_uri
                    }

model_matches = sm_client.list_models(NameContains=model_name)["Models"]
if not model_matches:
    model = sm_client.create_model(ModelName=model_name,
                                   PrimaryContainer=primary_container,
                                   ExecutionRoleArn=sagemaker_role)
else:
    print(f"Model with name {model_name} already exists! Change model name to create new")

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Cannot create already existing model "arn:aws:sagemaker:us-east-1:720025708473:model/fraud-detect-xgb".

In [14]:
# Endpoint Config name
endpoint_config_name = f"{model_name}-endpoint-config"

# Endpoint config parameters
production_variant_dict = {
                           "VariantName": "Alltraffic",
                           "ModelName": model_name,
                           "InitialInstanceCount": 1,
                           "InstanceType": "ml.m5.xlarge",
                           "InitialVariantWeight": 1
                          }

# Data capture config parameters
data_capture_config_dict = {
                            "EnableCapture": True,
                            "InitialSamplingPercentage": 100,
                            "DestinationS3Uri": data_capture_uri,
                            "CaptureOptions": [{"CaptureMode" : "Input"}, {"CaptureMode" : "Output"}]
                           }


# Create endpoint config if one with the same name does not exist
endpoint_config_matches = sm_client.list_endpoint_configs(NameContains=endpoint_config_name)["EndpointConfigs"]
if not endpoint_config_matches:
    endpoint_config_response = sm_client.create_endpoint_config(
                                                                EndpointConfigName=endpoint_config_name,
                                                                ProductionVariants=[production_variant_dict],
                                                                DataCaptureConfig=data_capture_config_dict
                                                               )
else:
		print(f"Endpoint config with name {endpoint_config_name} already exists! Change endpoint config name to create new")

Endpoint config with name fraud-detect-xgb-endpoint-config already exists! Change endpoint config name to create new


In [15]:
endpoint_name = f"{model_name}-endpoint"

endpoint_matches = sm_client.list_endpoints(NameContains=endpoint_name)["Endpoints"]
if not endpoint_matches:
    endpoint_response = sm_client.create_endpoint(
                                                  EndpointName=endpoint_name,
                                                  EndpointConfigName=endpoint_config_name
                                                 )
else:
    print(f"Endpoint with name {endpoint_name} already exists! Change endpoint name to create new")

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Endpoint Status: {status}...")
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
print(f"Endpoint Status: {status}")

Endpoint with name fraud-detect-xgb-endpoint already exists! Change endpoint name to create new
Endpoint Status: InService


In [16]:
# Fetch test data to run predictions with the endpoint
test_df = pd.read_csv(test_data_uri)

# For content type text/csv, payload should be a string with commas separating the values for each feature
# This is the inference request serialization step
# CSV serialization
csv_file = io.StringIO()
test_sample = test_df.drop(["fraud"], axis=1).iloc[:5]
test_sample.to_csv(csv_file, sep=",", header=False, index=False)
payload = csv_file.getvalue()
response = sm_runtime_client.invoke_endpoint(
                                             EndpointName=endpoint_name,
                                             Body=payload,
                                             ContentType="text/csv",
                                             Accept="text/csv"
                                            )

# This is the inference response deserialization step
# This is a bytes object
result = response["Body"].read()
# Decoding bytes to a string
result = result.decode("utf-8")
# Converting to list of predictions
result = re.split(",|\n",result)

prediction_df = pd.DataFrame()
prediction_df["Prediction"] = result[:5]
prediction_df["Label"] = test_df["fraud"].iloc[:5].values
prediction_df 


/opt/conda/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,Prediction,Label
0,0.02243666537106037,0
1,0.02243666537106037,0
2,0.0799826979637146,0
3,0.13931523263454437,0
4,0.03112351894378662,0


In [17]:
test_df.head()

,fraud,num_vehicles_involved,num_injuries,num_witnesses,police_report_available,injury_claim,vehicle_claim,total_claim_amount,incident_month,incident_day,...,authorities_contacted_ambulance,policy_state_ca,policy_state_az,policy_state_nv,policy_state_id,policy_state_wa,policy_state_or,customer_gender_other,customer_gender_male,customer_gender_female
0,0,2,0,0,1,58400,14247.766867,72647.766867,7,29,...,0,1,0,0,0,0,0,0,0,1
1,0,2,0,2,0,11500,10675.671347,22175.671347,11,28,...,0,1,0,0,0,0,0,0,0,1
2,0,2,0,0,1,18500,10202.266354,28702.266354,1,6,...,0,1,0,0,0,0,0,0,1,0
3,0,1,0,0,0,16300,9338.348066,25638.348066,9,1,...,0,1,0,0,0,0,0,0,1,0
4,0,2,1,0,0,14700,28145.924994,42845.924994,10,23,...,1,0,0,0,0,0,1,0,1,0


In [18]:
from sagemaker.s3 import S3Downloader
print("Waiting for captures to show up", end="")
for _ in range(90):
    capture_files = sorted(S3Downloader.list(f"{data_capture_uri}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    time.sleep(1)
print()
print(f"Found {len(capture_files)} Data Capture Files:")

Waiting for captures to show up..........................................................................................
Found 20 Data Capture Files:


In [19]:
capture_files = sorted(S3Downloader.list(f"{data_capture_uri}/{endpoint_name}"))
capture_file = S3Downloader.read_file(capture_files[0]).split("\n")
capture_record = json.loads(capture_file[0])
capture_record

{'captureData': {'endpointInput': {'observedContentType': 'text/csv',
   'mode': 'INPUT',
   'data': 'MiwwLDAsMSw1ODQwMCwxNDI0Ny43NjY4NjY2NzA4NSw3MjY0Ny43NjY4NjY2NzA4NSw3LDI5LDAsMTEsNDEsMTA4LDAsMSw3NTAsMzAwMCwwLDIsMjAxNCwwLDAsMSwwLDAsMSwwLDAsMCwxLDAsMCwwLDAsMSwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMCwxCjIsMCwyLDAsMTE1MDAsMTA2NzUuNjcxMzQ2NzM4MTU4LDIyMTc1LjY3MTM0NjczODE2LDExLDI4LDMsMTYsNTQsMTcxLDAsMSw3NTAsMjc1MCwyLDMsMjAxNSwwLDAsMCwwLDEsMSwwLDAsMCwxLDAsMCwwLDAsMSwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMCwxCjIsMCwwLDEsMTg1MDAsMTAyMDIuMjY2MzUzOTk5Nzc2LDI4NzAyLjI2NjM1Mzk5OTc3LDEsNiw2LDgsNTEsMTk2LDAsMSw3NTAsMzAwMCwwLDEsMjAxMSwwLDAsMCwwLDEsMSwwLDAsMSwwLDAsMCwwLDEsMCwwLDAsMSwwLDEsMCwwLDAsMCwwLDAsMSwwCjEsMCwwLDAsMTYzMDAsOTMzOC4zNDgwNjY0MzU3MTgsMjU2MzguMzQ4MDY2NDM1NzIsOSwxLDYsMyw1NywxMjIsMCwxLDc1MCwzMDAwLDEsMSwyMDE1LDAsMCwwLDEsMCwwLDEsMCwwLDAsMSwwLDAsMCwxLDAsMCwxLDAsMSwwLDAsMCwwLDAsMCwxLDAKMiwxLDAsMCwxNDcwMCwyODE0NS45MjQ5OTQxODIyNSw0Mjg0NS45MjQ5OTQxODIyNSwxMCwyMywyLDE1LDU5LDE0MiwwLDEsNzUwLDMwMDAsMiwzLDIwMTksMCwwLD

In [20]:
input_data = capture_record["captureData"]["endpointInput"]["data"]
output_data = capture_record["captureData"]["endpointOutput"]["data"]
input_data_list = base64.b64decode(input_data).decode("utf-8").split("\n")
print(input_data_list)
output_data_list = base64.b64decode(output_data).decode("utf-8").split("\n")
print(output_data_list)

['2,0,0,1,58400,14247.76686667085,72647.76686667085,7,29,0,11,41,108,0,1,750,3000,0,2,2014,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1', '2,0,2,0,11500,10675.671346738158,22175.67134673816,11,28,3,16,54,171,0,1,750,2750,2,3,2015,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1', '2,0,0,1,18500,10202.266353999776,28702.26635399977,1,6,6,8,51,196,0,1,750,3000,0,1,2011,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0', '1,0,0,0,16300,9338.348066435718,25638.34806643572,9,1,6,3,57,122,0,1,750,3000,1,1,2015,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0', '2,1,0,0,14700,28145.92499418225,42845.92499418225,10,23,2,15,59,142,0,1,750,3000,2,3,2019,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0', '']
['0.02243666537106037', '0.02243666537106037', '0.0799826979637146', '0.13931523263454437', '0.03112351894378662', '']


In [21]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)

# SageMaker expects resource id to be provided with the following structure
resource_id = f"endpoint/{endpoint_name}/variant/{resp['ProductionVariants'][0]['VariantName']}"

# Scaling configuration
scaling_config_response = sm_autoscaling_client.register_scalable_target(
                                                          ServiceNamespace="sagemaker",
                                                          ResourceId=resource_id,
                                                          ScalableDimension="sagemaker:variant:DesiredInstanceCount", 
                                                          MinCapacity=1,
                                                          MaxCapacity=2
                                                        )

In [22]:
# Create Scaling Policy
policy_name = f"scaling-policy-{endpoint_name}"
scaling_policy_response = sm_autoscaling_client.put_scaling_policy(
                                                PolicyName=policy_name,
                                                ServiceNamespace="sagemaker",
                                                ResourceId=resource_id,
                                                ScalableDimension="sagemaker:variant:DesiredInstanceCount",
                                                PolicyType="TargetTrackingScaling",
                                                TargetTrackingScalingPolicyConfiguration={
                                                    "TargetValue": 5.0, # Target for avg invocations per minutes
                                                    "PredefinedMetricSpecification": {
                                                        "PredefinedMetricType": "SageMakerVariantInvocationsPerInstance",
                                                    },
                                                    "ScaleInCooldown": 600, # Duration in seconds until scale in
                                                    "ScaleOutCooldown": 60 # Duration in seconds between scale out
                                                }
                                            )

In [23]:
response = sm_autoscaling_client.describe_scaling_policies(ServiceNamespace="sagemaker")

pp = pprint.PrettyPrinter(indent=4, depth=4)
for i in response["ScalingPolicies"]:
    pp.pprint(i["PolicyName"])
    print("")
    if("TargetTrackingScalingPolicyConfiguration" in i):
        pp.pprint(i["TargetTrackingScalingPolicyConfiguration"])

'scaling-policy-DeepSeek-Coder-V2-Instruct-2024-08-26-18-13-10-225'

{   'PredefinedMetricSpecification': {   'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'},
    'ScaleInCooldown': 600,
    'ScaleOutCooldown': 60,
    'TargetValue': 5.0}
'scaling-policy-fraud-detect-xgb-endpoint'

{   'PredefinedMetricSpecification': {   'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance'},
    'ScaleInCooldown': 600,
    'ScaleOutCooldown': 60,
    'TargetValue': 5.0}


In [24]:
request_duration = 250
end_time = time.time() + request_duration
print(f"Endpoint will be tested for {request_duration} seconds")
while time.time() < end_time:
    csv_file = io.StringIO()
    test_sample = test_df.drop(["fraud"], axis=1).iloc[[np.random.randint(0, test_df.shape[0])]]
    test_sample.to_csv(csv_file, sep=",", header=False, index=False)
    payload = csv_file.getvalue()
    response = sm_runtime_client.invoke_endpoint(
                                                 EndpointName=endpoint_name,
                                                 Body=payload,
                                                 ContentType="text/csv"
                                                )

Endpoint will be tested for 250 seconds


In [ ]:
# print(endpoint_name)
# test_df.shape[0]
# test_sample.info()

In [25]:
test_sample.head()

,num_vehicles_involved,num_injuries,num_witnesses,police_report_available,injury_claim,vehicle_claim,total_claim_amount,incident_month,incident_day,incident_dow,...,authorities_contacted_ambulance,policy_state_ca,policy_state_az,policy_state_nv,policy_state_id,policy_state_wa,policy_state_or,customer_gender_other,customer_gender_male,customer_gender_female
44,3,1,3,1,1400,36372.867146,37772.867146,11,7,3,...,0,1,0,0,0,0,0,0,1,0


In [26]:
# Check the instance counts after the endpoint gets more load
response = sm_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_status = response["EndpointStatus"]
request_duration = 250
end_time = time.time() + request_duration
print(f"Waiting for Instance count increase for a max of {request_duration} seconds. Please re run this cell in case the count does not change")
while time.time() < end_time:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_status = response["EndpointStatus"]
    instance_count = response["ProductionVariants"][0]["CurrentInstanceCount"]
    print(f"Status: {endpoint_status}")
    print(f"Current Instance count: {instance_count}")
    if (endpoint_status=="InService") and (instance_count>1):
        break
    else:
        time.sleep(15)

Waiting for Instance count increase for a max of 250 seconds. Please re run this cell in case the count does not change
Status: InService
Current Instance count: 1
Status: InService
Current Instance count: 1
Status: InService
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 1
Status: Updating
Current Instance count: 2
Status: InService
Current Instance count: 2
